In [5]:
class Station:
    import notebook_importer
    from Logger      import Logger
    nb_d_instance  = 0
    station_logger = Logger("./stations.csv")
    output_dir     = "./Stations/"

    def __init__(self, position):
        import notebook_importer
        import random
        from Velo   import Velo
        from Logger import Logger
        import os
        Station.nb_d_instance +=1
        NB_BORNES_POSSIBLES = [5,10,15,20,30]
        self.position       = position
        self.nb_bornes      = random.choice(NB_BORNES_POSSIBLES)
        self.velos          = [Velo() for i in range(self.nb_bornes-3)]
        self.id             = Station.nb_d_instance
        
        self.recupere_velo_log   = Logger(os.path.join(Station.output_dir, "recupere_velo_log_stations_%s.csv"%self.id))
        self.donne_velo_log      = Logger(os.path.join(Station.output_dir,    "donne_velo_log_stations_%s.csv"%self.id))

        log_format  = "timestamp,station_id,cycliste_id,velo_id,velo_performance"
        self.recupere_velo_log.log(log_format)
        self.donne_velo_log.log(log_format)


    def __str__(self):
        to_print = "Station n° {nom} : {position}, {nb_bornes} bornes, {nb_velos_dispo} velos"

        return to_print.format(nom=self.id,
                               position=self.position,
                              nb_bornes=self.nb_bornes,
                              nb_velos_dispo=len(self.velos))
    def nb_de_velos(self):
        return len(self.velos)

    def has_empty_place(self):
        return len(self.velos) < self.nb_bornes

    def has_velo(self):
        return 0 < len(self.velos)

    def log(self, message):
        message = "station %s, %s"%(self.id, message)
        Station.station_logger.log(message)
    def get_log_message(self, timestamp, cycliste, velo):
        return ",".join([str(timestamp), str(self.id), str(cycliste.id), str(velo.id), str(velo.performance)])
        
    def deposer_un_velo(self, timestamp, cycliste, velo):
        if self.has_empty_place():
            message = self.get_log_message(timestamp, cycliste, velo)
            self.recupere_velo_log.log(message)
            velo.update_performance(timestamp)
            self.velos.append(velo)
            return True
        else:
            self.log("complete")
            return False
    def prendre_un_velo(self, timestamp, cycliste):
        if self.has_velo():
            import random
            velo    = self.velos.pop(random.randint(0, len(self.velos)-1))
            message = self.get_log_message(timestamp, cycliste, velo)
            self.donne_velo_log.log(message)
            return velo
        else:
            self.log("vide.")
            return False

    def nb_velo_under_perf(self, seuil_de_performance=0.5):
        test = lambda velo: velo.performance < seuil_de_performance
        return len(list(filter(test, self.velos)))
